In [1]:
%%writefile configs/selfsup/relative_loc/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab.py
_base_ = [
    '../_base_/models/relative-loc.py',
    '../_base_/datasets/imagenet_relative-loc.py',
    '../_base_/schedules/sgd_steplr-200e_in1k.py',
    '../_base_/default_runtime.py',
]

default_hooks = dict(logger=dict(type='LoggerHook', interval=10))

# optimizer wrapper
optimizer = dict(type='SGD', lr=0.2, momentum=0.9, weight_decay=1e-4)
optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=optimizer,
    paramwise_cfg=dict(custom_keys={
        'neck': dict(decay_mult=5.0),
        'head': dict(decay_mult=5.0)
    }))

# learning rate scheduler
param_scheduler = [dict(type='MultiStepLR', by_epoch=True, milestones=[1, 2])]

# runtime settings
# pre-train for 70 epochs
train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=70)
# the max_keep_ckpts controls the max number of ckpt file in your work_dirs
# if it is 3, when CheckpointHook (in mmcv) saves the 4th ckpt
# it will remove the oldest one to keep the number of total ckpts as 3
default_hooks = dict(
    checkpoint=dict(type='CheckpointHook', interval=1, max_keep_ckpts=3))

Writing configs/selfsup/relative_loc/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab.py


In [2]:
# Load the basic config file
from mmengine.config import Config
cfg = Config.fromfile('configs/selfsup/relative_loc/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab.py')

# Specify the data settings
cfg.train_dataloader.batch_size = 8
cfg.train_dataloader.num_workers = 2

# Specify the optimizer
cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
cfg.optim_wrapper.clip_grad = None

# Specify the learning rate scheduler
cfg.param_scheduler = [dict(type='MultiStepLR', by_epoch=True, milestones=[1, 2])]

# Modify runtime setting
cfg.train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=2)

# Specify the work directory
cfg.work_dir = './work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab'

# Output logs for every 10 iterations
cfg.default_hooks.logger.interval = 10
# Set the random seed and enable the deterministic option of cuDNN
# to keep the results' reproducible.
cfg.randomness = dict(seed=0, deterministic=True)

In [3]:
from mmengine.runner import Runner
from mmengine.analysis import get_model_complexity_info
# build the runner from config
runner = Runner.from_cfg(cfg)
input_shape = (3, 3, 10, 10)
analysis_results = get_model_complexity_info(runner.model, input_shape)
print(analysis_results['out_table'])

08/28 21:02:00 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.17 (default, Jul  5 2023, 21:04:15) [GCC 11.2.0]
    CUDA available: False
    numpy_random_seed: 0
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.1
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINETO -


+--------------------------+----------------------+------------+--------------+
| module                   | #parameters or shape | #flops     | #activations |
+--------------------------+----------------------+------------+--------------+
| model                    | 40.33M               | 89.51M     | 0.155M       |
|  backbone                |  23.508M             |  89.51M    |  0.155M      |
|   backbone.conv1         |   9.408K             |   0.706M   |   4.8K       |
|    backbone.conv1.weight |    (64, 3, 7, 7)     |            |              |
|   backbone.bn1           |   0.128K             |   24K      |   0          |
|    backbone.bn1.weight   |    (64,)             |            |              |
|    backbone.bn1.bias     |    (64,)             |            |              |
|   backbone.layer1        |   0.216M             |   5.941M   |   38.016K    |
|    backbone.layer1.0     |    75.008K           |    2.077M  |    17.28K    |
|    backbone.layer1.1     |    70.4K  

In [4]:
runner.train()

08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.fc.weight:lr=0.2
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.fc.weight:weight_decay=0.0005
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.fc.weight:decay_mult=5.0
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.fc.bias:lr=0.2
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.fc.bias:weight_decay=0.0005
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.fc.bias:decay_mult=5.0
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.bn.weight:lr=0.2
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.bn.weight:weight_decay=0.0005
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.bn.weight:decay_mult=5.0
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.bn.bias:lr=0.2
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.bn.bias:weight_decay=0.0005
08/28 21:03:03 - mmengine - INFO - paramwise_options -- neck.bn.bias:decay_m

KeyboardInterrupt: 